In [10]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import os
import pickle

# Paths
ratings_path = "D:/movie_recommendation_system/rcmndn_model/data/processed/ratings_final.csv"
movies_path = "D:/movie_recommendation_system/rcmndn_model/data/processed/movies_cleaned.csv"
save_path = "D:/movie_recommendation_system/rcmndn_model/trained_models/item_similarity.pkl"

# 1. Load data
ratings = pd.read_csv(ratings_path)
movies = pd.read_csv(movies_path)

# 2. Create user–movie matrix
user_movie_matrix = ratings.pivot(index="userId", columns="movieId", values="rating").fillna(0)
print(f" User-Movie matrix created: {user_movie_matrix.shape}")

# 3. Compute cosine similarity between movies
item_similarity = cosine_similarity(user_movie_matrix.T)  # movies × movies
item_similarity_df = pd.DataFrame(
    item_similarity,
    index=user_movie_matrix.columns,
    columns=user_movie_matrix.columns
)
print(f" Item-Item similarity matrix created: {item_similarity_df.shape}")

# 4. Save similarity matrix
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with open(save_path, "wb") as f:
    pickle.dump(item_similarity_df, f)

print(f" Item similarity matrix saved at: {save_path}")

# 5. Function to recommend movies
def recommend_movies_cf(movie_id, top_n=5):
    if movie_id not in item_similarity_df.index:
        return pd.DataFrame({"Message": [f"Movie ID {movie_id} not found in dataset"]})

    # Get similarity scores
    sim_scores = item_similarity_df[movie_id].sort_values(ascending=False)

    # Drop the same movie
    sim_scores = sim_scores.drop(movie_id)

    # Top N
    top_movies = sim_scores.head(top_n).index
    return movies[movies["movieId"].isin(top_movies)][["movieId", "title", "genres", "vote_average"]]

# --- Quick test ---
test_movie_id = 1  # Toy Story (1995)
print(f"\n🎬 Collaborative Filtering Recommendations for '{movies[movies['movieId']==test_movie_id]['title'].values[0]}':")
print(recommend_movies_cf(test_movie_id, top_n=5))


 User-Movie matrix created: (610, 2187)
 Item-Item similarity matrix created: (2187, 2187)
 Item similarity matrix saved at: D:/movie_recommendation_system/rcmndn_model/trained_models/item_similarity.pkl

🎬 Collaborative Filtering Recommendations for 'Toy Story (1995)':
     movieId                                      title  \
80       260  Star Wars: Episode IV - A New Hope (1977)   
103      356                        Forrest Gump (1994)   
128      480                       Jurassic Park (1993)   
187      780       Independence Day (a.k.a. ID4) (1996)   
865     3114                         Toy Story 2 (1999)   

                                          genres  vote_average  
80                       Action|Adventure|Sci-Fi           8.1  
103                     Comedy|Drama|Romance|War           8.2  
128             Action|Adventure|Sci-Fi|Thriller           7.6  
187             Action|Adventure|Sci-Fi|Thriller           6.7  
865  Adventure|Animation|Children|Comedy|Fantasy 